# Add Folders to Path

In [1]:
%%time
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])

# add Algorithm folder to current working path in order to access the functions
sys.path.append(parent_directory+"/Src")

CPU times: user 186 µs, sys: 150 µs, total: 336 µs
Wall time: 195 µs


In [2]:
# Suppress numba warnings arising from computation falling back into object mode.
from numba.core.errors import NumbaDeprecationWarning
import warnings

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)

# Import Data

In [3]:
%%time
import scipy.io as sio

#Import velocity data from file in data-folder
mat_file = sio.loadmat('../../Data/Agulhas_AVISO.mat')

U = mat_file['u']
V = mat_file['v']
x = mat_file['x']
y = mat_file['y']
time = mat_file['t']

#mat_file = sio.loadmat('../../Data/ocean_velocity_example_diffusive.mat')

#U = mat_file['vx'].transpose()
#V = mat_file['vy'].transpose()
#x = mat_file['xc'].transpose()
#y = mat_file['yc'].transpose()
#time = mat_file['time'].transpose()

CPU times: user 17.2 ms, sys: 12.1 ms, total: 29.3 ms
Wall time: 30 ms


# Data/Parameters for Dynamical System

In [4]:
import numpy as np

# number of cores to be used for parallel computing
Ncores = 50

# periodic boundary conditions
periodic_x = False
periodic_y = False
Periodic = [periodic_x, periodic_y]

## compute meshgrid of dataset
X, Y = np.meshgrid(x, y)

# list of parameters
params_data = {"X": X, "Y": Y, "Time": time, "U": U, "V": V, "Ncores": Ncores, "Periodic": Periodic}

# Spatio-Temporal Domain of Dynamical System

In [5]:
%%time
# Initial time (in days)
t0 = 25

# Final time (in days)
tN = 45

# time step-size
dt = .1

time = np.arange(t0, tN+dt, dt)

# domain boundaries (in degrees)
xmin = -3
xmax = 1
ymin = -32
ymax = -24

# spacing of meshgrid (in degrees)
dx = 0.02
dy = 0.02

x_domain = np.arange(xmin, xmax + dx, dx)
y_domain = np.arange(ymin, ymax + dy, dy)

# meshgrid of domain
X_domain, Y_domain = np.meshgrid(x_domain, y_domain)

# Define parameters
params_DS = {"time": time, "X_domain": X_domain, "Y_domain": Y_domain}

# Initialize Dynamical System
from ipynb.fs.defs.DynamicalSystem import *

DS = Dynamical_System(params_data, params_DS)

CPU times: user 116 ms, sys: 32.9 ms, total: 148 ms
Wall time: 148 ms


# Interpolate Velocity

In order to evaluate the velocity field at arbitrary locations and times, we must interpolate the discrete velocity data. The interpolation with respect to time is always linear. The interpolation with respect to space can be chosen to be "cubic" or "linear". In order to favour a smooth velocity field, we interpolate the velocity field in space using a cubic interpolant.

In [6]:
%%time
# Interpolate velocity data using cubic spatial interpolation
DS._Interpolation_velocity("cubic")

CPU times: user 95.1 ms, sys: 6.36 ms, total: 101 ms
Wall time: 101 ms


# Cauchy Green (CG) strain tensor over meshgrid of initial conditions

The rate of strain tensor $ S(\mathbf{x}, t) $ at time $ t $ is computed by iterating over meshgrid. The method *DS._spin_tensor(x, t)* computes the rate of strain tensor at point $ \mathbf{x} $  at time $ t $ by using an auxiliary meshgrid. 'aux_grid' specifies the ratio between the auxiliary grid and the original meshgrid. This parameter is generally chosen to be between $ [\dfrac{1}{20}, \dfrac{1}{100}] $. The computations are parallelized.

In [7]:
%%time
# Import function to calculate Cauchy Green (CG) strain tensor from gradient of Flow map
from ipynb.fs.defs.CauchyGreen import _CauchyGreen

# tqdm is used to display progress bar
from tqdm.notebook import tqdm

# joblib is used for parallel computing
from joblib import Parallel, delayed

# Define ratio of auxiliary grid vs original meshgrid
aux_grid = 0.1

def parallel(i):
    
    # Define CG tensor
    C_ = np.zeros((X_domain.shape[1], 2, 2))
    
    # Iterate over initial conditions
    for j in range(X_domain.shape[1]):
        
        # Point at which to calculate CG tensor
        x = np.array([X_domain[i, j], Y_domain[i, j]])
        
        # Calculate gradient of flow map at 'x' over the time-interval 'time' using an auxiliary grid ratio 'aux_grid'
        grad_Fmap = DS._grad_Fmap(x, time, aux_grid)
        
        C_[j, :, :] = _CauchyGreen(grad_Fmap)
        
    return C_

C = np.array(Parallel(n_jobs=Ncores, verbose = 0)(delayed(parallel)(i) for i in tqdm(range(X_domain.shape[0]))))

  0%|          | 0/401 [00:00<?, ?it/s]

IndexError: index 1 is out of bounds for axis 0 with size 1

# Compute Tensorfield Properties

We now compute the properties of the rate of strain tensor 'S' such as the eigenvalues 's1', 's2' and eigenvectors 'eigenv1', 'eigenv2'. Furthermore, we also need the spatial derivatives of the elements of 'S'.

In [8]:
from ipynb.fs.defs.tensorfield import _tensorfield

# Compute tensorfield properties
s1, s2, eigenv1, eigenv2, C11, C12, C22, C11x, C11y, C12x, C12y, C22x, C22y = _tensorfield(X_domain, Y_domain, C, aux_grid, type = 'LCS')

NameError: name 'C' is not defined

# Interpolate $ \dot{\phi}(x, y, \phi)$ 

In [ ]:
%%time
from ipynb.fs.defs.phi_prime import _phi_prime

# Interpolant phi_phrime
interp_phi_prime, interp_DOE = _phi_prime(X_domain, Y_domain, C11, C12, C22, C11x, C11y, C12x, C12y, C22x, C22y, Ncores)

# Find closed null-geodesics of $ C_{t_0}^t − \lambda^2 I$

Elliptic OECSs are closed null geodesics of the one-parameter family of Lorentzian metrics $ C_{t_0}^t − \lambda^2 I, \lambda ∈ \mathbb{R} $. 

In [ ]:
%%time
# function which computes closed null geodesics
from ipynb.fs.defs.closed_null_geodesics import _closed_null_geodesics
# function which computes outermost closed null geodesics
from ipynb.fs.defs.outermost_elliptic_LCS import _outermost_elliptic_LCS

# define lambda range
lambda_min = 0.9
lambda_max = 1.1
n_lambda = 5
lambda_range = np.linspace(lambda_min, lambda_max, n_lambda)

# define threshold distance for periodicity of trajectory
d_threshold = 3*dx

# parameter specifying sparsity of initial conditions.
# If sparse_ic = 1, then original meshgrid is used for computation of initial level sets.
# Increasing this parameter reduces the number of initial conditions and thereby reduces the computational time involved.
sparse_ic = 2

elliptic_LCS = []

for lam in lambda_range:
    
    print("Compute elliptic LCS for lambda =", lam)
    
    # Define list storing x, y coordinates of elliptic OECS
    x_elliptic, y_elliptic = [], []
    
    # Find all closed null geodesics
    closed_null_geodesics = _closed_null_geodesics(X_domain, Y_domain, lam, interp_phi_prime, d_threshold, C11, interp_DOE, sparse_ic, Ncores=1)
    
    for elliptic in closed_null_geodesics:
        
        x_elliptic.append(elliptic[0])
        y_elliptic.append(elliptic[1])    
        
    # Extract outermost closed null geodesics
    outermost_elliptic_LCS = _outermost_elliptic_LCS(closed_null_geodesics)
    
    # Store outermost closed null geodesics in list
    if outermost_elliptic_LCS:
        
        for elliptic in outermost_elliptic_LCS:
            
            x_elliptic.append(elliptic[0])
            y_elliptic.append(elliptic[1])
            
    elliptic_LCS.append([x_elliptic, y_elliptic])

In [ ]:
######################################## Plot elliptic OECS ########################################

# Define figure/axes
fig = plt.figure(figsize = (16, 10), dpi = 300)
ax = plt.axes()

# Define norm/cmap
norm = mpl.colors.Normalize(vmin=mu_range.min(), vmax=mu_range.max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.jet)

# Iterate over all elliptic LCS 
for i in range(len(elliptic_LCS)):
    
    for j in range(len(elliptic_LCS[i][0])):
        
        ax.plot(elliptic_LCS[i][0][j], elliptic_LCS[i][1][j], c=cmap.to_rgba(mu_range[i]), linewidth = 3)

# Set axis limits
ax.set_xlim([xmin, xmax])
ax.set_ylim([ymin, ymax])

# Set axis labels
ax.set_xlabel("long (deg)", fontsize = 16)
ax.set_ylabel("lat (deg)", fontsize = 16)

# Colorbar
cbar = fig.colorbar(cmap, ticks=[mu_range.min(), 0, mu_range.max()])
cbar.ax.set_ylabel(r'$ \mu $', rotation = 0, fontsize = 16)

# Title
ax.set_title("Elliptic LCS", fontsize = 20)
plt.show();